In [19]:
# DROWSINESS DETECTION USING DLIB

# Eye Aspect Ratio = EAR < 0.26 -> DROWSY

#notCorrectDf
#(15057, 5)
#CorrectDf
#(26736, 5)

#ACCURACY = 26736/41793 = 0.6397243557533558

In [20]:
import pandas as pd
import re
import os
import numpy as np
import dlib
import cv2
import requests
import bz2


In [21]:
def download_and_decompress(url, local_path):
    # Download the file
    response = requests.get(url)
    with open(local_path + '.bz2', 'wb') as file:
        file.write(response.content)

    # Decompress the file
    with bz2.BZ2File(local_path + '.bz2', 'rb') as file:
        content = file.read()
    with open(local_path, 'wb') as file:
        file.write(content)

# Define file paths
datFile = "shape_predictor_68_face_landmarks.dat"
cnnFile = "mmod_human_face_detector.dat"

# Download and decompress files
if not os.path.exists(datFile):
    download_and_decompress('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2', datFile)

if not os.path.exists(cnnFile):
    download_and_decompress('http://dlib.net/files/mmod_human_face_detector.dat.bz2', cnnFile)

# Initialize Dlib models
import dlib
hog_face_detector = dlib.get_frontal_face_detector()
cnn_detector = dlib.cnn_face_detection_model_v1(cnnFile)
dlib_facelandmark = dlib.shape_predictor(datFile)


In [22]:
current_directory = os.getcwd()
data_dir = os.path.join(current_directory, 'Driver Drowsiness Dataset (DDD)')

In [28]:
# DEMO OF DLIB FACE DETECTION OF ONE IMAGE WITH CNN_DETECTOR

# Combine the current directory path with the image filename
testimagepath = os.path.join(data_dir, 'Non Drowsy/d0590.png')

#testimagepath = '/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/Non Drowsy/d0590.png'
testimage = cv2.imread(os.path.join(testimagepath),cv2.IMREAD_UNCHANGED)

#faceList = hog_face_detector(testimage)
faceList = cnn_detector(testimage)

for singleFace in faceList:
  face_landmarks = dlib_facelandmark(testimage, singleFace.rect)
  
  #draw point for every face_landmark point
  for n in range(0, 68):
    x = face_landmarks.part(n).x
    y = face_landmarks.part(n).y
    cv2.circle(testimage, (x, y), 1, (0, 255, 0), 1)
cv2.imshow("test image", testimage)
cv2.waitKey(0) # Wait for a key press to close the window
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to target thread (0x555ea68bd800)

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to target thread (0x555ea68bd800)

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to target thread (0x555ea68bd800)

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to target thread (0x555ea68bd800)

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to target thread (0x555ea68bd800)

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to target thread (0x555ea68bd800)

QObject::moveToThread: Current thread (0x555ea68bd800) is not the object's thread (0x555eabc11d40).
Cannot move to tar

# Neuer Abschnitt

In [29]:
from scipy.spatial import distance

#calculate the eye aspect ratio
def calculate_EAR(eye):
  A = distance.euclidean(eye[1], eye[5])
  B = distance.euclidean(eye[2], eye[4])
  C = distance.euclidean(eye[0], eye[3])
  eye_aspect_ratio = (A+B)/(2.0*C)
  return eye_aspect_ratio


In [30]:
# FUNCTION ADDS A PREDICTION DROWSY OR NON_DROWSY TO PREDICTIONS LIST

def predictLabelWithEAR(dlib_facelandmark, faces, grayscale, predictions):
  #go through all faces that have been detected by the hog_face_detector
  for face in faces:
    face_landmarks = dlib_facelandmark(grayscale, face.rect)
    leftEye = []
    rightEye = []

    #save landmarks of the eyes
    #leftEye
    for n in range(36, 42):
      x = face_landmarks.part(n).x
      y = face_landmarks.part(n).y
      #cv2.circle(image, (x, y), 1, (0, 0, 255), 1)
      leftEye.append((x,y))
    
    #rightEye
    for n in range(42, 48):
      x = face_landmarks.part(n).x
      y = face_landmarks.part(n).y
      #cv2.circle(image, (x, y), 1, (0, 255, 0), 1)
      rightEye.append((x,y))
    
    #calculate eye aspect ratio for every eye
    left_ear = calculate_EAR(leftEye)
    right_ear = calculate_EAR(rightEye)

    ear = (left_ear+right_ear)/2
    ear = round(ear, 2)

    #one face detected in the image
    if len(faces)==1:
      #if eye aspect ratio is below a threshold - classify as drowsy
      if ear < 0.26:
        #drowsy
        predictions.append(0)
        return 0
      else:
        #not drowsy
        predictions.append(1)
        return 0
    else:
      #multiple faces detected in the image
      predictions.append(2)
      return 0


In [32]:
# CREATE A DATAFRAME OF ALL PICTURES IN THE DATASET WITH REAL LABEL AND PREDICTED LABEL BY EAR

# Add image paths, labels, file names and person IDs into lists
image_path, label, file_name, person_ID = [], [], [], []

#list of predicted labels by the eye aspect ratio
predictions = []

#for the person_ID - identification criteria of participant
pattern = re.compile(r'^[a-zA-Z][a-zA-Z]?')

#data_dir = "/content/drive/MyDrive/newdata/Driver Drowsiness Dataset (DDD)/"
#data_dir = "/content/drive/MyDrive/smalltest/testDDD/"

for class_name in os.listdir(data_dir):  #goes through all folders in DDD folder
  for imageName in os.listdir(data_dir+'/'+class_name):  #goes through all files in both folders and updates file path and labels
    #correct label
    if class_name == 'Drowsy':
      label.append(0)
    else:
      label.append(1)
    #image path
    image_path.append(os.path.join(data_dir, class_name, imageName))
    #file name
    file_name.append(imageName)
    #personID
    person_ID.append(pattern.findall(imageName)[0])  #identification of participant

    #prediction of the label by landmark
    image = cv2.imread(os.path.join(data_dir, class_name, imageName),cv2.IMREAD_UNCHANGED)
    #grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #faces = hog_face_detector(image)
    faces = cnn_detector(image)
    if len(faces)==0:
      #no face was detected
      predictions.append(-1)
    else:
      #face was detected - predict the label
      predictLabelWithEAR(dlib_facelandmark, faces, image, predictions)

#dataframe with path, label, predicted label, file name, person_ID of all Persons
df = pd.DataFrame()
df['images'] = image_path
df['label'] = label
df['predlabel'] = predictions
df['name'] = file_name
df['person'] = person_ID

print('df')
print(df)
print('df shape')
print(df.shape)

df
                                                  images  label  predlabel  \
0      /home/maroof/TechLabs/TechLabs_DriverDrowsines...      0          0   
1      /home/maroof/TechLabs/TechLabs_DriverDrowsines...      0          0   
2      /home/maroof/TechLabs/TechLabs_DriverDrowsines...      0          0   
3      /home/maroof/TechLabs/TechLabs_DriverDrowsines...      0          0   
4      /home/maroof/TechLabs/TechLabs_DriverDrowsines...      0          0   
...                                                  ...    ...        ...   
41788  /home/maroof/TechLabs/TechLabs_DriverDrowsines...      1          1   
41789  /home/maroof/TechLabs/TechLabs_DriverDrowsines...      1          1   
41790  /home/maroof/TechLabs/TechLabs_DriverDrowsines...      1          1   
41791  /home/maroof/TechLabs/TechLabs_DriverDrowsines...      1          1   
41792  /home/maroof/TechLabs/TechLabs_DriverDrowsines...      1          0   

            name person  
0      P0092.png      P  
1      T

In [33]:
# COMPARISON CLASSIFIED WRONG VS CLASSIFIED CORRECTLY

#classiefied wrong by eye aspect ratio
notCorrectDf = df[df['label']!=df['predlabel']]
print('notCorrectDf')
print(notCorrectDf.shape)

#classified correctly by eye aspect ratio
CorrectDf = df[df['label']==df['predlabel']]
print('CorrectDf')
print(CorrectDf.shape)

notCorrectDf
(15057, 5)
CorrectDf
(26736, 5)


In [34]:
accuracy = CorrectDf.shape[0]/df.shape[0]
print('accuracy')
print(accuracy)

accuracy
0.6397243557533558


In [35]:
notCorrectDf.groupby('person').size()

person
A      350
B       65
C      326
D       88
E      955
F      395
G       16
H      494
I      125
J        1
K      450
L      242
M        5
N       13
O      859
P       20
Q      210
R      168
S       37
T       78
U      225
V       76
W      978
X      313
Y       64
ZA      47
ZB     783
ZC    1327
a      189
b       20
c        4
d      186
e       21
g        8
h       94
i      730
j       68
k       40
l        6
m      380
n      658
o        6
p      187
q       13
r       38
s       56
u       24
v      670
w        2
x     1103
y      518
za    1016
zb     280
zc      30
dtype: int64

In [36]:
#error face couldnt be detected
noFaceDetection = notCorrectDf[notCorrectDf['predlabel']==-1]
print('noFaceDetection')
print(noFaceDetection.shape)
print('-----')
noFaceDetection.groupby('person').size()

noFaceDetection
(961, 5)
-----


person
E     908
G       5
U       1
ZB     25
c       1
e      21
dtype: int64

In [37]:
#face detected but classified wrong
classifiedWrong = notCorrectDf[notCorrectDf['predlabel']!=-1]

classifiedWrong = classifiedWrong[classifiedWrong['predlabel']!=2]

print('classifiedWrong')
print(classifiedWrong.shape)
print('-----')
classifiedWrong.groupby('person').size()

classifiedWrong
(14061, 5)
-----


person
A      350
B       65
C      326
D       88
E       47
F      395
G       11
H      487
I      125
J        1
K      450
L      242
M        5
N       13
O      859
P       20
Q      210
R      168
S       37
T       51
U      224
V       76
W      978
X      313
Y       64
ZA      47
ZB     758
ZC    1327
a      189
b       20
c        3
d      186
g        8
h       94
i      730
j       68
k       40
l        6
m      380
n      658
o        6
p      187
q       13
r       38
s       56
u       24
v      670
w        2
x     1103
y      518
za    1015
zb     280
zc      30
dtype: int64

In [38]:
#correct labels of the wrong classified
classifiedWrong.groupby('label').size()

#try out ear < 0.27

label
0    7737
1    6324
dtype: int64

In [39]:
#multiple faces detected but only one face per image
multipleFacesDetected = notCorrectDf[notCorrectDf['predlabel']==2]
print('multipleFacesDetected')
print(multipleFacesDetected.shape)
print('-----')
multipleFacesDetected.groupby('person').size()

multipleFacesDetected
(35, 5)
-----


person
H      7
T     27
za     1
dtype: int64

In [40]:
accuracy = CorrectDf.shape[0]/df.shape[0]
print('accuracy')
print(accuracy)

accuracy
0.6397243557533558
